In [1]:
import numpy as np
import time
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis
import random
import math
import warnings
import keras

In [5]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

In [6]:
def predictEnsemble(ensemble, X):
    X = X.reshape(1, -1)
    predictions = [model.predict(X) for model in ensemble]
    predictions = np.array(predictions)
    predictionsFlipped = predictions.transpose()
    output = []
    for pred in predictionsFlipped:
        mean = pred.mean()
        interval = 1.96 * pred.std()
        lower, upper = mean - interval, mean + interval
        output.append([round(lower), round(mean), round(upper)])
    return output

In [7]:
def dtwFast(x1, y1, x2, y2, window):
    n, m = len(x1), len(x2)
    w = window
    dtw_matrix = np.zeros((n+1, m+1))
    
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        ofset = round(i*(m/n))
        for j in range(np.max([1, ofset-w]), np.min([m, ofset+w])+1):
            cost = math.sqrt(((x2[j-1]-x1[i-1])**2)+((y2[j-1]-y1[i-1])**2))#*sample
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix    

def get_key(val, my_dict):
    for key, value in my_dict.items():
         if val == value:
             return key

percentErrors = []
percentErrors2 = []
numCorrect = 0
numCorrect2 = 0
numTrials = 0
subTimes1 = []
times1 = []
times2 = []
times3 = []


warnings.filterwarnings('ignore')

length = 60
width = 60
length -= 1
width -= 1
width2 = 11
width2 -= 1

def performDTW(x1, x2, y1, y2, ensemble, scaler):
    n = len(x1)
    m = len(x2)
    if n < 230 or m < 230:
        return 0, 0, 0, [0], 0, 0, 0, 0
    
    x1Samp = [0]*230
    x2Samp = [0]*230
    y1Samp = [0]*230
    y2Samp = [0]*230
    for i in range(len(x1)):
        x1Samp[230*i//len(x1)] = x1[i]
    for i in range(len(x2)):
        x2Samp[230*i//len(x2)] = x2[i]
    for i in range(len(y1)):
        y1Samp[230*i//len(y1)] = y1[i]
    for i in range(len(y2)):
        y2Samp[230*i//len(y2)] = y2[i]
    
    x1SampMax = max(abs(max(x1Samp)), abs(min(x1Samp)))
    x2SampMax = max(abs(max(x2Samp)), abs(min(x2Samp)))
    y1SampMax = max(abs(max(y1Samp)), abs(min(y1Samp)))
    y2SampMax = max(abs(max(y2Samp)), abs(min(y2Samp)))
    for i in range(len(x1Samp)):
        x1Samp[i] /= x1SampMax
    for i in range(len(x2Samp)):
        x2Samp[i] /= x2SampMax
    for i in range(len(y1Samp)):
        y1Samp[i] /= y1SampMax
    for i in range(len(y2Samp)):
        y2Samp[i] /= y2SampMax

    x1, x2, y1, y2 = np.asarray(x1Samp), np.asarray(x2Samp), np.asarray(y1Samp), np.asarray(y2Samp)
    n = len(x1)
    m = len(x2)
    ### ml matrix

    startTime = time.time()

    dtw_matrix = np.zeros((n+1, m+1))
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0


    mlinput = []

    mlinput += x1.tolist()[0:length] + y1.tolist()[0:length] + x2.tolist()[0:width] + y2.tolist()[0:width]
    confidence = []
    mlinput = np.array(mlinput)        
    mlinput = mlinput.reshape(1, -1)
    waypoints = []
    waypoints.append(0)
    confidence.append(0.25)
    
    mlStartTime = time.time()
    transformedModelInput = scaler.transform(mlinput)
    
    output = predictEnsemble(ensemble, transformedModelInput)
    for lower, mean, upper in output:
        waypoints.append(mean)
        confidence.append(upper-lower)
    waypoints.append(n)
    confidence.append(0.5)

    mlTime = time.time() - mlStartTime

    for i in range(len(waypoints)):
        waypoints[i] = round(waypoints[i])
    path = [(0, 0)]

    waypointCols = []

    l = []
    for i in range(0, m, m//3):
        l.append(i)
    if len(l) == 3:
        l.append(2)
    if len(l) == 4:
        l.append(m-2)
    l.sort()

    for i in l:
        waypointCols.append(i)

    waypointCols.append(m)

    for i in range(1, len(waypoints)):
        wp0 = waypoints[i]
        curRow, curCol = path[-1][0], path[-1][1]
        wp1 = waypoints[i]
        targetRow = min(max(curRow+1, wp0), n-1)
        targetCol = waypointCols[i]
        if (targetCol - curCol) <= 0:
            while curRow != targetRow:
                path.append((curRow, curCol))
                curRow += 1
        else:
            slope = (targetRow - curRow) / (targetCol - curCol)
            slope = max(0, slope)
            lastRow = curRow
            for i in range(targetCol - curCol):
                path.append((lastRow, (i + curCol)))
                if i == targetCol - curCol - 1:
                    for j in range(lastRow, targetRow):
                        path.append((j, (i + curCol)))
                        lastRow = j
                else:
                    for j in range(lastRow, round(i*slope)+curRow):
                        path.append((j, (i + curCol)))
                        lastRow = j
                    
        best_path = []
        for i in path:
            if i not in best_path:
                best_path.append(i)
    # confidence widths #
    confidenceWidths = []
    widths = []
    for val in confidence:
        confidenceWidths.append(min(60, round((val+1)*12)))
    for i in range(len(confidenceWidths)-1):
        widths.append(confidenceWidths[i])
        slope = (confidenceWidths[i+1]-confidenceWidths[i])/(n//3)
        for j in range(n//3-1):
            widths.append(int(confidenceWidths[i]+slope*j))
    widths.append(confidenceWidths[len(confidenceWidths)-1])
    widths.append(confidenceWidths[len(confidenceWidths)-1])
    subStartTime = time.time()

    for i in range(1, m+1):
        valsInCol = []
        middleVal = 0
        for xVal, yVal in best_path:
            if yVal+1 == i:
                valsInCol.append(yVal)
        if len(valsInCol) != 0:
            middleVal = sum(valsInCol) // len(valsInCol)
        width2 = widths[i-1]
        if len(valsInCol) >= width2:
            width2 = len(valsInCol) + 1

        if middleVal-(width2//2) < 0:
            xi = 1
            xf = min(n, xi+width2)
        else:
            xf = min(n, middleVal+(width2//2+1))
            xi = xf-width2
        if i == 1:
            xi = 1
            xf = min(n, xi+width2)
        elif i == m:
            xf = n
            xi = xf-width2
        for j in range(xi, xf+1):
            if dtw_matrix[j, i] == np.inf:
                cost = math.sqrt(((x2[i-1]-x1[j-1])**2)+((y2[i-1]-y1[j-1])**2))
                last_min = np.min([dtw_matrix[j-1, i], dtw_matrix[j, i-1], dtw_matrix[j-1, i-1]])
                dtw_matrix[j, i] = cost + last_min

    dist1 = dtw_matrix[len(dtw_matrix)-1][len(dtw_matrix[0])-1]
    endTime = time.time()
    elapsedTime1 = endTime - startTime
    subElaspedTime1 = endTime - subStartTime
    best_path1 = dtw.best_path(dtw_matrix)
    #dtwvis.plot_warpingpaths(x1, x2, dtw_matrix, best_path1)
    ### full matrix
    startTime = time.time()
    dtw_matrix2 = np.zeros((n+1, m+1))
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix2[i, j] = np.inf
    dtw_matrix2[0, 0] = 0

    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = math.sqrt(((x2[j-1]-x1[i-1])**2)+((y2[j-1]-y1[i-1])**2))
            last_min = np.min([dtw_matrix2[i-1, j], dtw_matrix2[i, j-1], dtw_matrix2[i-1, j-1]])
            dtw_matrix2[i, j] = cost + last_min
    
    endTime = time.time()
    elapsedTime2 = endTime - startTime

    dist2 = dtw_matrix2[len(dtw_matrix2)-1][len(dtw_matrix2[0])-1]
    best_path2 = dtw.best_path(dtw_matrix2)
    #dtwvis.plot_warpingpaths(x1, x2, dtw_matrix2, best_path2)

    # fast matrix
    startTime = time.time()
    dtw_matrix3 = dtwFast(x1, y1, x2, y2, 14)
    endTime = time.time()
    elapsedTime3 = endTime - startTime
    dist3 = dtw_matrix3[len(dtw_matrix3)-1][len(dtw_matrix3[0])-1]
    best_path3 = dtw.best_path(dtw_matrix3)
    #dtwvis.plot_warpingpaths(x1, x2, dtw_matrix3, best_path3)
    #plt.show()
    return dist1, dist2, dist3, best_path1, x1, x2, y1, y2
    

In [8]:
character = "signature"

In [9]:
import csv
xTrain = []
yTrain = []
with open(f'{character}Data.csv', newline='') as csvfile: #signatureData, letterAData.csv
    csv_reader = csv.reader(csvfile, delimiter=',')
    counter = 0
    for line in csv_reader:
        for i in range(len(line)):
            line[i] = float(line[i])
        if counter % 2 == 0:
            xTrain.append(line) 
        else:
            yTrain.append(line)
        counter += 1


In [34]:
# application signature iPad Version

from math import pi
import numpy as np
from ipycanvas import MultiCanvas
import copy
import matplotlib.pyplot as plt
from pickle import load
from keras.src.legacy.saving import legacy_h5_format

# ml full fast
canvas  = MultiCanvas(2, width=1600, height=1200, layout=dict(width="100%"))

signatureEnsemble = []
for i in range(5):
    # signatureEnsemble.append(keras.models.load_model(f'{character}_{i}.h5'))
    signatureEnsemble.append(legacy_h5_format.load_model_from_hdf5(f'{character}_{i}.h5', custom_objects={'mae': 'mae'}))

signatureScaler = load(open(f'scaler_{character}.pkl', 'rb'))

from ipywidgets import Output

drawOn = False
letterX = []
letterY = []
canvas[0]
xPos = 0
yPos = 0

letterR_X = xTrain[15]
letterR_Y = yTrain[15]
for i in range(len(letterR_X)):
    canvas[0].fill_arc(letterR_X[i]+100, -1*(letterR_Y[i])+500, 5, 0, 180)
canvas[0].fill_rect(50, 1000, 1500, 5)

canvas[1]  
out = Output()
@out.capture()
def handle_mouse_move(x, y):
    global drawOn
    global letterX
    global letterY
    global xPos
    global yPos
    xPos = x
    yPos = y
    if drawOn:
        canvas[1].fill_arc(x, y, 5, 0, 180)
        letterX.append(x)
        letterY.append(-1*(y))

canvas.on_mouse_move(handle_mouse_move)

@out.capture()
def on_keyboard_event(key, shift_key, ctrl_key, meta_key):
    global drawOn
    global letterX
    global letterY
    global xPos
    global yPos
    global letterR_X
    global letterR_Y
    if key == " ":
        drawOn = not drawOn
        if drawOn:
            return 0
        startX = letterX[0]
        startY = letterY[0]
        for i in range(len(letterX)):
            letterX[i] -= startX
            letterY[i] -= startY
        dist1, dist2, dist3, best_path, x1, x2, y1, y2 = performDTW(letterR_X, letterX, letterR_Y, letterY, signatureEnsemble, signatureScaler)
        if dist1 == 0:
            print("need more points")
        else:
            avgdist1 = 0
            avgdist2 = 0
            avgdist3 = 0
            passes = 10
            
            for i in range(passes):
                sample = random.randint(0, len(xTrain)-1)
                dist1, dist2, dist3, valbest_path, valx1, valx2, valy1, valy2 = performDTW(xTrain[sample], letterX, yTrain[sample], letterY, signatureEnsemble, signatureScaler)
                percentError = (abs(dist1 - dist2) / dist2) * 100
                percentErrors.append(percentError)

                percentError2 = (abs(dist3 - dist2) / dist2) * 100
                percentErrors2.append(percentError2)

                avgdist1 += dist1
                avgdist2 += dist2
                avgdist3 += dist3
            avgdist1 /= passes
            avgdist2 /= passes
            avgdist3 /= passes
            plt.plot(x1, y1, 'o', color='blue')
            plt.plot(x2, y2, 'o', color='red')
            counter = 0
            for a, b in best_path:
                if counter % 1 == 0:
                    point1 = [x1[a], y1[a]]
                    point2 = [x2[b], y2[b]]
                    x_values = [point1[0], point2[0]]
                    y_values = [point1[1], point2[1]]
                    plt.plot(x_values, y_values, 'g', linestyle="--") 
                counter += 1
            plt.show()

            if avgdist1 < 25:
                print("correct")
            elif 25 < avgdist1 < 32:
                print("maybe correct")
            else:
                print("incorrect")
            print("MLDTW:", avgdist1, "Full DTW:", avgdist2, "FastDTW:", avgdist3)
        letterX = []
        letterY = []
        canvas[1].clear()


@out.capture()
def handle_mouse_down(x, y):
    global drawOn
    global letterX
    global letterY
    
canvas.on_key_down(on_keyboard_event)
canvas.on_mouse_down(handle_mouse_down)
display(out)

display(canvas)

Output()

MultiCanvas(height=1200, layout=Layout(width='100%'), width=1600)

In [13]:
# application signature

from math import pi
import numpy as np
from ipycanvas import MultiCanvas
import copy
import matplotlib.pyplot as plt
from pickle import load
from keras.src.legacy.saving import legacy_h5_format

# ml full fast
canvas  = MultiCanvas(2, width=1600, height=1200, layout=dict(width="100%"))

signatureEnsemble = []
for i in range(5):
    # signatureEnsemble.append(keras.models.load_model(f'{character}_{i}.h5'))
    signatureEnsemble.append(legacy_h5_format.load_model_from_hdf5(f'{character}_{i}.h5', custom_objects={'mae': 'mae'}))

signatureScaler = load(open(f'scaler_{character}.pkl', 'rb'))

from ipywidgets import Output

drawOn = False
letterX = []
letterY = []
canvas[0]
xPos = 0
yPos = 0

letterR_X = xTrain[15]
letterR_Y = yTrain[15]

for i in range(len(letterR_X)):
    canvas[0].fill_arc(letterR_X[i]+100, -1*(letterR_Y[i])+500, 5, 0, 180)
canvas[0].fill_rect(50, 1000, 1500, 5)

canvas[1]  
out = Output()
@out.capture()
def handle_mouse_move(x, y):
    global drawOn
    global letterX
    global letterY
    global xPos
    global yPos
    xPos = x
    yPos = y
    if drawOn:
        canvas[1].fill_arc(x, y, 5, 0, 180)
        letterX.append(x)
        letterY.append(-1*(y))

canvas.on_mouse_move(handle_mouse_move)

@out.capture()
def on_keyboard_event(key, shift_key, ctrl_key, meta_key):
    global drawOn
    global letterX
    global letterY
    global xPos
    global yPos
    global letterR_X
    global letterR_Y
    if key == " ":
        startX = letterX[0]
        startY = letterY[0]
        for i in range(len(letterX)):
            letterX[i] -= startX
            letterY[i] -= startY
        dist1, dist2, dist3, best_path, x1, x2, y1, y2 = performDTW(letterR_X, letterX, letterR_Y, letterY, signatureEnsemble, signatureScaler)
        if dist1 == 0:
            print("need more points")
        else:
            avgdist1 = 0
            avgdist2 = 0
            avgdist3 = 0
            passes = 10
            
            for i in range(passes):
                sample = random.randint(0, len(xTrain)-1)
                dist1, dist2, dist3, valbest_path, valx1, valx2, valy1, valy2 = performDTW(xTrain[sample], letterX, yTrain[sample], letterY, signatureEnsemble, signatureScaler)
                percentError = (abs(dist1 - dist2) / dist2) * 100
                percentErrors.append(percentError)

                percentError2 = (abs(dist3 - dist2) / dist2) * 100
                percentErrors2.append(percentError2)

                avgdist1 += dist1
                avgdist2 += dist2
                avgdist3 += dist3
            avgdist1 /= passes
            avgdist2 /= passes
            avgdist3 /= passes
            plt.plot(x1, y1, 'o', color='blue')
            plt.plot(x2, y2, 'o', color='red')
            counter = 0
            for a, b in best_path:
                if counter % 1 == 0:
                    point1 = [x1[a], y1[a]]
                    point2 = [x2[b], y2[b]]
                    x_values = [point1[0], point2[0]]
                    y_values = [point1[1], point2[1]]
                    plt.plot(x_values, y_values, 'g', linestyle="--") 
                counter += 1
            plt.show()

            if avgdist1 < 25:
                print("correct")
            elif 25 < avgdist1 < 32:
                print("maybe correct")
            else:
                print("incorrect")
            print("MLDTW:", avgdist1, "Full DTW:", avgdist2, "FastDTW:", avgdist3)
        letterX = []
        letterY = []
        canvas[1].clear()
        drawOn = False


@out.capture()
def handle_mouse_down(x, y):
    global drawOn
    global letterX
    global letterY
    drawOn = not drawOn
    
canvas.on_key_down(on_keyboard_event)
canvas.on_mouse_down(handle_mouse_down)
display(out)

display(canvas)

Output()

MultiCanvas(height=1200, layout=Layout(width='100%'), width=1600)

In [14]:
# set base signal

letterR_X = []
letterR_Y = []

from math import pi
import numpy as np
from ipycanvas import MultiCanvas
import copy
import matplotlib.pyplot as plt

# ml full fast
canvas  = MultiCanvas(2, width=1600, height=1200, layout=dict(width="100%"))

from ipywidgets import Output

drawOn = False
letterX = []
letterY = []
startX = 0
startY = 0
canvas[0]
for i in range(len(xTrain[10])):
    canvas[0].fill_arc(xTrain[10][i], -1*(yTrain[10][i])-500, 5, 0, 180)

canvas[1]  
out = Output()
@out.capture()
def handle_mouse_move(x, y):
    global drawOn
    global letterX
    global letterY
    global startX
    global startY
    if drawOn:
        canvas[1].fill_arc(x, y, 5, 0, 180)
        letterX.append(x-startX)
        letterY.append(-1*(y-startY))

canvas.on_mouse_move(handle_mouse_move)

@out.capture()
def handle_mouse_down(x, y):
    global drawOn
    global letterX
    global letterY
    global startX
    global startY
    global letterR_X
    global letterR_Y
    if drawOn:
        letterR_X = letterX
        letterR_Y = letterY

        dist1, dist2, dist3, best_path, x1, x2, y1, y2 = performDTW(xTrain[10], letterX, yTrain[10], letterY)
        plt.plot(x1, y1, 'o', color='blue')
        plt.plot(x2, y2, 'o', color='red')
        counter = 0
        for a, b in best_path:
            if counter % 1 == 0:
                point1 = [x1[a], y1[a]]
                point2 = [x2[b], y2[b]]
                x_values = [point1[0], point2[0]]
                y_values = [point1[1], point2[1]]
                plt.plot(x_values, y_values, 'g', linestyle="--") 
            counter += 1
        plt.show()

        if dist1 < 15000:
            print("r")
        else:
            print("not r")
        print("MLDTW:", dist1, "Full DTW:", dist2, "FastDTW:", dist3)
        letterX = []
        letterY = []
        canvas[1].clear()
    else:
         startX = x
         startY = y
    drawOn = not drawOn
    

canvas.on_mouse_down(handle_mouse_down)
display(out)

display(canvas)


Output()

MultiCanvas(height=1200, layout=Layout(width='100%'), width=1600)

In [2]:
# collect data

import csv
from math import pi
import numpy as np
from ipycanvas import MultiCanvas
import copy
import matplotlib.pyplot as plt

# ml full fast
canvas  = MultiCanvas(2, width=1600, height=1200, layout=dict(width="100%"))

from ipywidgets import Output

drawOn = False
letterX = []
letterY = []
canvas[0]
xPos = 0
yPos = 0

canvas[1]  
out = Output()
@out.capture()
def handle_mouse_move(x, y):
    global drawOn
    global letterX
    global letterY
    global xPos
    global yPos
    xPos = x
    yPos = y
    if drawOn:
        canvas[1].fill_arc(x, y, 5, 0, 180)
        letterX.append(x)
        letterY.append(-1*(y))

canvas.on_mouse_move(handle_mouse_move)

@out.capture()
def on_keyboard_event(key, shift_key, ctrl_key, meta_key):
    global drawOn
    global letterX
    global letterY
    global xPos
    global yPos
    if key == " ":
        startX = letterX[0]
        startY = letterY[0]
        for i in range(len(letterX)):
            letterX[i] -= startX
            letterY[i] -= startY
        file = open(f'letterAData.csv','a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(letterX)
            writer.writerow(letterY)
        letterX = []
        letterY = []
        canvas[1].clear()
        drawOn = False

        print('Keyboard event:', key, shift_key, ctrl_key, meta_key)

@out.capture()
def handle_mouse_down(x, y):
    global drawOn
    global letterX
    global letterY
    drawOn = not drawOn
    
canvas.on_key_down(on_keyboard_event)
canvas.on_mouse_down(handle_mouse_down)
display(out)

display(canvas)


Output()

MultiCanvas(height=1200, layout=Layout(width='100%'), width=1600)

In [10]:
from sklearn.model_selection import train_test_split

import math
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis
import numpy as np
import csv
import os

def myround(x, base):
    return base * round(x/base)

label = ""
#character = "signature"

xTrain, xTest, yTrain, yTest = train_test_split(xTrain, yTrain, test_size=0.1,  random_state=1)

length = 60 #10 for a's
width = 60 #10 for a's
length -= 1
width -= 1

file = f'handwriting_{character}_StartWave.csv'
if(os.path.exists(file) and os.path.isfile(file)):
    os.remove(file)

header = ""
# for i in range(length):
#     for j in range(width):
#         header += f"col {i}: {j}, "
for i in range(length):
    header += f"x1_{i}, "
for i in range(length):
    header += f"y1_{i}, "
for j in range(width):
    header += f"x2_{j}, "
for j in range(width):
    header += f"y2_{j}, "
for i in range(4):
    header += f"waypoints {i}, "

header = header.split(", ")
header.pop()
file = open(f'handwriting_{character}_StartWave.csv','w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

for n1 in range(len(xTrain)):
    for n2 in range(len(xTrain)):
        if n1 == n2:
            continue
        
        x1, x2, y1, y2 = xTrain[n1], xTrain[n2], yTrain[n1], yTrain[n2]
        minLength = 230 #35 for a's, 230 for sig
        n = len(x1)
        m = len(x2)
        if n < minLength or m < minLength:
            continue
        
        x1Samp = [0]*minLength
        x2Samp = [0]*minLength
        y1Samp = [0]*minLength
        y2Samp = [0]*minLength
        for i in range(len(x1)):
            x1Samp[minLength*i//len(x1)] = x1[i]
        for i in range(len(x2)):
            x2Samp[minLength*i//len(x2)] = x2[i]
        for i in range(len(y1)):
            y1Samp[minLength*i//len(y1)] = y1[i]
        for i in range(len(y2)):
            y2Samp[minLength*i//len(y2)] = y2[i]

        x1SampMax = max(abs(max(x1Samp)), abs(min(x1Samp)))
        x2SampMax = max(abs(max(x2Samp)), abs(min(x2Samp)))
        y1SampMax = max(abs(max(y1Samp)), abs(min(y1Samp)))
        y2SampMax = max(abs(max(y2Samp)), abs(min(y2Samp)))
        for i in range(len(x1Samp)):
            x1Samp[i] /= x1SampMax
        for i in range(len(x2Samp)):
            x2Samp[i] /= x2SampMax
        for i in range(len(y1Samp)):
            y1Samp[i] /= y1SampMax
        for i in range(len(y2Samp)):
            y2Samp[i] /= y2SampMax

        x1, x2, y1, y2 = np.asarray(x1Samp), np.asarray(x2Samp), np.asarray(y1Samp), np.asarray(y2Samp)
        n = len(x1)
        m = len(x2)
        dtw_matrix = np.zeros((n+1, m+1))
        for i in range(n+1):
            for j in range(m+1):
                dtw_matrix[i, j] = np.inf
        dtw_matrix[0, 0] = 0
        for i in range(1, n+1):
            for j in range(1, m+1):
                cost = math.sqrt(((x2[j-1]-x1[i-1])**2)+((y2[j-1]-y1[i-1])**2))
                last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
                dtw_matrix[i, j] = cost + last_min
        dist = dtw_matrix[len(dtw_matrix)-1][len(dtw_matrix[0])-1]
        best_path = dtw.best_path(dtw_matrix)
        
        dtw_matrix = np.zeros((n+1, m+1))
        for i in range(n+1):
            for j in range(m+1):
                dtw_matrix[i, j] = np.inf
        dtw_matrix[0, 0] = 0

        line = []

        line += x1.tolist()[0:length] + y1.tolist()[0:length] + x2.tolist()[0:width] + y2.tolist()[0:width]

        waypoints = []
        l = []
        for i in range(0, m, m//3):
            l.append(i)
        if len(l) == 3:
            l.append(2)
        if len(l) == 4:
            l.append(m-2)
        l.sort()
        for i in l:
            vals = []
            for row, col in best_path:
                if col == i:
                    vals.append(row)
            waypoints.append((sum(vals)//len(vals), i))
        waypoints.pop(0)
        for wprow, wpcol in waypoints:
            line.append(wprow)
        if len(waypoints) == 4:
            file = open(f'handwriting_{character}_StartWave.csv','a', newline='')
            with file:
                writer = csv.writer(file)
                writer.writerow(line)

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras 
import warnings
from pickle import dump


data = pd.read_csv(f'handwriting_{character}_StartWave.csv')

encoder = LabelEncoder()
y = np.array(data.iloc[:, -4:])
for i in range(4):
    data=data.drop([f"waypoints {i}"],axis=1)
#for i in range(30,199):
#    data=data.drop([f"x1_{i}"],axis=1)
#for i in range(30,199):
 #   data=data.drop([f"x2_{i}"],axis=1)
scaler = StandardScaler()
array = np.array(data.iloc[:])
#array = array.astype(int)
scaler.fit(array)
X = scaler.transform(np.array(data.iloc[:]))
dump(scaler, open(f'scaler_{character}.pkl', 'wb'))
X_train_ML, X_test_ML, y_train_ML, y_test_ML = train_test_split(X, y, test_size=0.1,  random_state=1)

In [28]:
from sklearn.metrics import mean_absolute_error

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

def fitModel(X_train, y_train):
    n_inputs, n_outputs = X_train.shape[1], y_train.shape[1]
    early_stopping = EarlyStopping(monitor='loss', patience=40)
    model = models.Sequential()
    model.add(layers.Dense(200,  activation='relu', kernel_initializer='he_uniform', input_shape =(n_inputs,)))
    model.add(layers.Dense(n_outputs, kernel_initializer='he_uniform'))
    model.compile(optimizer='adam', loss='mae', metrics =['accuracy'])
    model.fit(X_train, y_train, epochs=500, callbacks=[early_stopping])
    return model

def fitEnsemble(n_members, X_train, X_test, y_train, y_test):
    ensemble = list()
    for i in range(n_members):
        model = fitModel(X_train, y_train)
        yhat = model.predict(X_test, verbose=0)
        mae = mean_absolute_error(y_test, yhat)
        print('>%d, MAE: %.3f' % (i+1, mae))
        ensemble.append(model)
        model.save(f'{character}_{i}.h5')
    return ensemble

X_train_ML, X_test_ML, y_train_ML, y_test_ML = train_test_split(X, y, test_size=0.1,  random_state=1)
ensemble = fitEnsemble(5, X_train_ML, X_test_ML, y_train_ML, y_test_ML)

Epoch 1/500
9/9 [==============================] - 0s 7ms/step - loss: 170.2754 - accuracy: 0.5091
Epoch 2/500
9/9 [==============================] - 0s 5ms/step - loss: 167.8727 - accuracy: 0.4836
Epoch 3/500


2022-04-27 16:13:49.080263: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 0s 5ms/step - loss: 165.1169 - accuracy: 0.4473
Epoch 4/500
9/9 [==============================] - 0s 5ms/step - loss: 161.6414 - accuracy: 0.4691
Epoch 5/500
9/9 [==============================] - 0s 5ms/step - loss: 157.1598 - accuracy: 0.4982
Epoch 6/500
9/9 [==============================] - 0s 5ms/step - loss: 151.4306 - accuracy: 0.5018
Epoch 7/500
9/9 [==============================] - 0s 5ms/step - loss: 144.2603 - accuracy: 0.5273
Epoch 8/500
9/9 [==============================] - 0s 5ms/step - loss: 135.2248 - accuracy: 0.5491
Epoch 9/500
9/9 [==============================] - 0s 5ms/step - loss: 124.5601 - accuracy: 0.5564
Epoch 10/500
9/9 [==============================] - 0s 5ms/step - loss: 112.9064 - accuracy: 0.5891
Epoch 11/500
9/9 [==============================] - 0s 5ms/step - loss: 101.4674 - accuracy: 0.6364
Epoch 12/500
9/9 [==============================] - 0s 5ms/step - loss: 89.1718 - accuracy: 0.6473
Epoch 13/500
9/9 [==

2022-04-27 16:14:02.854847: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-27 16:14:03.027196: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 0s 8ms/step - loss: 170.3008 - accuracy: 0.5018
Epoch 2/500
9/9 [==============================] - 0s 5ms/step - loss: 167.9234 - accuracy: 0.4618
Epoch 3/500
9/9 [==============================] - 0s 5ms/step - loss: 165.1664 - accuracy: 0.4145
Epoch 4/500
9/9 [==============================] - 0s 5ms/step - loss: 161.7142 - accuracy: 0.3964
Epoch 5/500
9/9 [==============================] - 0s 5ms/step - loss: 157.1478 - accuracy: 0.4036
Epoch 6/500
9/9 [==============================] - 0s 5ms/step - loss: 151.4569 - accuracy: 0.4109
Epoch 7/500
9/9 [==============================] - 0s 5ms/step - loss: 144.2241 - accuracy: 0.4509
Epoch 8/500
9/9 [==============================] - 0s 5ms/step - loss: 135.2445 - accuracy: 0.4727
Epoch 9/500
9/9 [==============================] - 0s 5ms/step - loss: 124.5214 - accuracy: 0.5236
Epoch 10/500
9/9 [==============================] - 0s 5ms/step - loss: 112.5735 - accuracy: 0.5636
Epoch 11/500
9/9 [===

...

2022-04-27 16:14:24.315942: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-27 16:14:24.474417: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 0s 7ms/step - loss: 170.3014 - accuracy: 0.5055
Epoch 2/500
9/9 [==============================] - 0s 5ms/step - loss: 167.9171 - accuracy: 0.4836
Epoch 3/500
9/9 [==============================] - 0s 10ms/step - loss: 165.2045 - accuracy: 0.4327
Epoch 4/500
9/9 [==============================] - 0s 5ms/step - loss: 161.7820 - accuracy: 0.4545
Epoch 5/500
9/9 [==============================] - 0s 5ms/step - loss: 157.3464 - accuracy: 0.4764
Epoch 6/500
9/9 [==============================] - 0s 5ms/step - loss: 151.6618 - accuracy: 0.5200
Epoch 7/500
9/9 [==============================] - 0s 5ms/step - loss: 144.4467 - accuracy: 0.5418
Epoch 8/500
9/9 [==============================] - 0s 5ms/step - loss: 135.5577 - accuracy: 0.5600
Epoch 9/500
9/9 [==============================] - 0s 5ms/step - loss: 124.7331 - accuracy: 0.5745
Epoch 10/500
9/9 [==============================] - 0s 5ms/step - loss: 112.8447 - accuracy: 0.5818
Epoch 11/500
9/9 [==

2022-04-27 16:14:37.435868: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-27 16:14:37.594234: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 0s 7ms/step - loss: 170.2963 - accuracy: 0.5236
Epoch 2/500
9/9 [==============================] - 0s 5ms/step - loss: 167.8874 - accuracy: 0.4764
Epoch 3/500
9/9 [==============================] - 0s 5ms/step - loss: 165.1298 - accuracy: 0.4291
Epoch 4/500
9/9 [==============================] - 0s 5ms/step - loss: 161.7030 - accuracy: 0.4400
Epoch 5/500
9/9 [==============================] - 0s 5ms/step - loss: 157.2313 - accuracy: 0.4291
Epoch 6/500
9/9 [==============================] - 0s 5ms/step - loss: 151.4657 - accuracy: 0.4873
Epoch 7/500
9/9 [==============================] - 0s 5ms/step - loss: 144.2216 - accuracy: 0.5200
Epoch 8/500
9/9 [==============================] - 0s 5ms/step - loss: 135.2207 - accuracy: 0.5309
Epoch 9/500
9/9 [==============================] - 0s 5ms/step - loss: 124.5394 - accuracy: 0.5491
Epoch 10/500
9/9 [==============================] - 0s 5ms/step - loss: 112.7279 - accuracy: 0.5745
Epoch 11/500
9/9 [===

2022-04-27 16:14:45.015445: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [ ]:
if dist1 != np.inf and dist2 != 0:
            percentError = (abs(dist1 - dist2) / dist2) * 100
            percentErrors.append(percentError)
            numTrials += 1

            percentError2 = (abs(dist3 - dist2) / dist2) * 100
            percentErrors2.append(percentError2)

            subTimes1.append(subElaspedTime1)
            times1.append(elapsedTime1)
            times2.append(elapsedTime2)
            times3.append(elapsedTime3)

            dtwvis.plot_warpingpaths(x1, x2, dtw_matrix, best_path1)
            dtwvis.plot_warpingpaths(x1, x2, dtw_matrix2, best_path2)
            dtwvis.plot_warpingpaths(x1, x2, dtw_matrix3, best_path3)

   
p1q1, p1q2, p1q3 = np.percentile(percentErrors,[25,50,75])

p2q1, p2q2, p2q3 = np.percentile(percentErrors2,[25,50,75])

t1q1, t1q2, t1q3 = np.percentile(times1,[25,50,75])
t2q1, t2q2, t2q3 = np.percentile(times2,[25,50,75])
t3q1, t3q2, t3q3 = np.percentile(times3,[25,50,75])
s1q1, s1q2, s1q3 = np.percentile(subTimes1,[25,50,75])

print("Num of Trials:", len(percentErrors))

print("ml:")
print("percent error:", "     q1:", p1q1, "     q2:", p1q2, "     q3:", p1q3)
print("time:", "     q1:", t1q1, "     q2:", t1q2, "     q3:", t1q3,"     ml subtime:", "     q1:", s1q1,"     q2:", s1q2,"     q3:", s1q3)
print("dtwfast:")
print("percent error:", "     q1:", p2q1, "     q2:", p2q2, "     q3:", p2q3)
print("time:", "     q1:", t3q1, "     q2:", t3q2, "     q3:", t3q3)
print("Full DTW:")
print("time:", "     q1:", t2q1, "     q2:", t2q2, "     q3:", t2q3)

print(percentErrors2)
print(percentErrors)
print(times2)
print(times3)
print(subTimes1)
print(times1)